## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

os.chdir("..")

### Imports

In [4]:
from src.video_processing import PoseEstimator, VideoProcessor
from src.features import JointSeries, AngleSeries, FourierSeries
from src.models import kNNClassifier, LSTMClassifier

## Load model

In [5]:
import torch

file_path = os.path.join("models", "daniel_knn_angles_model.pth")

knn = torch.load(file_path)

print(f"Loaded kNN model from {file_path}")

Loaded kNN model from models\daniel_knn_angles_model.pth


## Record video via webcam

In [6]:
pose_estimator = PoseEstimator(
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

VIDEO_INPUT_WEBCAM = 1

video_processor = VideoProcessor(
    pose_estimator=pose_estimator,
    webcam=VIDEO_INPUT_WEBCAM,
)

video_processor.process_video(
    show=True,
    height=800,
    width=None,
)

2023-08-18 18:09:36,101 - INFO - Initializing PoseEstimator: model_complexity=2
2023-08-18 18:09:36,126 - INFO - Initializing VideoProcessor for input: webcam (source 1)
[DONE] Body pose estimation: webcam (source 1): : 96it [00:09,  9.76it/s]     
2023-08-18 18:09:47,049 - INFO - Body pose estimation completed (96 frames, 6.40s).


## Extract features

In [7]:
joint_series = JointSeries(landmarks_series=video_processor.landmarks_series, fps=video_processor.fps)
angle_series = AngleSeries(joint_series=joint_series)
fourier_series = FourierSeries(angle_series=angle_series)


2023-08-18 18:09:47,243 - INFO - Extracting joint coordinates from landmarks
2023-08-18 18:09:47,296 - INFO - Calculating angles from joint coordinates
2023-08-18 18:09:51,059 - INFO - Computing Fourier transform of angle time series


## Model prediction

In [8]:
knn.predict(angle_series)

2023-08-18 18:09:54,130 - INFO - Predicting class labels for input data: 1 sample(s)


'pullups'

In [ ]:
lstm = LSTMClassifier(
    num_classes=len(set(y_train)),
    input_size=len(X_train[0].columns),
    hidden_size=32,
    num_layers=2,
    num_epochs=10,
    batch_size=8,
    learning_rate=0.001,
)